In [1]:
with open("inputs/Day_10.txt") as f:
    raw_input_data = f.read()

In [2]:
def part_1_solution(raw_input):
    points = parse_input(raw_input)
    
    los_cnt = dict()
    for point in points:
        # tg is periodic so coords system needs to be spit in half (180)
        angle_tg = {
            "right": set(),
            "left": set()
        }
        for another_point in points:
            if point == another_point:
                continue
                
            if point[1] == another_point[1]:
                #same y (row)
                tg = "inf"
            elif point[0] == another_point[0]:
                # same x (column)
                if point[1] > another_point[1]:
                    angle_tg["left"].add(0)
                else:
                    angle_tg["right"].add(0)
                continue
            else:
                tg = (point[0] - another_point[0])/(point[1] - another_point[1])
                
            if point[0] > another_point[0]:
                angle_tg["left"].add(tg)
            else:
                angle_tg["right"].add(tg)
                
        visible_points_cnt = len(angle_tg["left"]) + len(angle_tg["right"])
        if visible_points_cnt not in los_cnt:
            los_cnt[visible_points_cnt] = list()
        
        los_cnt[visible_points_cnt].append(point)
    
    return max(los_cnt)

def parse_input(raw_input):
    points = set()
    
    for j, row in enumerate(raw_input.split()):
        for i, element in enumerate(row):
            if element == "#":
                point = (i, j)
                points.add(point)
                
    return points 

In [3]:
raw_test_input = """.#..#
.....
#####
....#
...##""" 
assert(part_1_solution(raw_test_input) == 8)
raw_test_input = """......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####""" 
assert(part_1_solution(raw_test_input) == 33)
raw_test_input = """#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.""" 
assert(part_1_solution(raw_test_input) == 35)
raw_test_input = """.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..""" 
assert(part_1_solution(raw_test_input) == 41)
raw_test_input = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##""" 
assert(part_1_solution(raw_test_input) == 210)
print("Tests passed")

Tests passed


In [4]:
print(f"Part 1 solution: {part_1_solution(raw_input_data)}")

Part 1 solution: 326


In [5]:
from collections import defaultdict
import math

def part_2_solution(raw_input):
    points = parse_input(raw_input)
    
    points_los_map, los_cnt = build_los_map(points)
    
    station_point = los_cnt[max(los_cnt)]
    assert(len(station_point) == 1)
    station_point = station_point[0]
    station_los = points_los_map[station_point]
    
    ordered_points = list()
    # build list oredered by angle (clockwise) which contains points ordered by distance in
    # that direction
    for direction in station_los:
        dir_list = list()
        for tg_angle in sorted(station_los[direction].keys()):
            dir_list.append(sorted(station_los[direction][tg_angle]))
        
        ordered_points.extend(dir_list)
    
    # 'destroy' points and return when 200th point is encountered
    point_nbr = 1
    while True:
        for points_with_dist in ordered_points:    
            curr_point_with_dist = points_with_dist.pop(0)
            
            if point_nbr == 200:
                return curr_point_with_dist[1][0] * 100 + curr_point_with_dist[1][1]
            
            point_nbr += 1
            
        # delete empty lists
        ordered_points = [x for x in ordered_points if x]

        
def get_distance(point_1, point_2):
    return math.sqrt((point_1[0] - point_2[0])**2 + (point_1[1] - point_2[1])**2)


def build_los_map(points):
    points_los_map = dict()
    los_cnt = defaultdict(list)
    
    for point in points:
        quarter = {
            "up_right": defaultdict(list),
            "down_right": defaultdict(list),
            "down_left": defaultdict(list),
            "up_left": defaultdict(list)
        }
        
        for another_point in points:
            if point == another_point:
                continue
                
            point_with_distance = (get_distance(point, another_point), another_point)
            
            if point[1] == another_point[1]:
                #same y (row)
                tg = -math.inf
                if another_point[0] > point[0]:
                    quarter["down_right"][tg].append(point_with_distance)
                else:
                    quarter["up_left"][tg].append(point_with_distance)
                    
                continue
            elif point[0] == another_point[0]:
                # same x (column)
                if point[1] > another_point[1]:
                    quarter["up_right"][0].append(point_with_distance)
                else:
                    quarter["down_left"][0].append(point_with_distance)
                    
                continue
            else:
                # y is decreasing so * - to be consistent
                tg = (point[0] - another_point[0])/-(point[1] - another_point[1])
            
            if another_point[0] > point[0]:
                # right
                if another_point[1] < point[1]:
                    quarter["up_right"][tg].append(point_with_distance)
                else:
                    quarter["down_right"][tg].append(point_with_distance)
            else:
                # left
                if another_point[1] < point[1]:
                    quarter["up_left"][tg].append(point_with_distance)
                else:
                    quarter["down_left"][tg].append(point_with_distance)
                
        points_los_map[point] = quarter
        
        visible_points_cnt = sum([len(v) for q, v in quarter.items()])
        
        los_cnt[visible_points_cnt].append(point)
        
    return points_los_map, los_cnt

def parse_input(raw_input):
    points = set()
    
    for j, row in enumerate(raw_input.split()):
        for i, element in enumerate(row):
            if element == "#":
                point = (i, j)
                points.add(point)
                
    return points 

In [6]:
raw_test_input = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""
assert(part_2_solution(raw_test_input) == 802)
print("Test passed")

Test passed


In [7]:
print(f"Part 2 solution: {part_2_solution(raw_input_data)}")

Part 2 solution: 1623
